### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [23]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2,
                 max_depth=None,
                 sufficient_share=1.0,
                 criterion='gini',
                 max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - (l_c ** 2 / l_s + r_c ** 2 / r_s).sum(axis=1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return - np.sum((l_c / l_c + r_c / r_s) *
                        log(l_c / l_c + r_c / r_s))

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (np.max(l_c / (l_s + r_s), axis=1)
                    + np.max(r_c / (l_s + r_s), axis=1))

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[np.sqrt(n_feature)]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.log2(n_feature)]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x, y = self.__sort_samples(x, y)
        
        border = np.where(y[:-1] != y[1:])[0] + 1
        if not len(border):
            return np.inf, None

        cls_mat = np.zeros((border.shape[0], self.num_class))
        cls_mat[np.arange(border.shape[0]), y[border - 1]] = 1
        cls_inc = cls_mat * \
                (border - np.append(np.array([0]), border[:-1])).reshape(-1, 1)

        l_c = np.cumsum(cls_inc, axis=0)
        r_c = np.bincount(y, minlength=self.num_class) - l_c
        l_s = border.reshape(l_c.shape[0], 1)
        r_s = y.shape[0] - l_s

        g = self.G_function(l_c, l_s, r_c, r_s)
        idx = np.argmin(g)
        
        id_ = l_s[idx][0]
        return g[idx], (x[id_ - 1] + x[id_]) / 2.0

    def __fit_node(self, x, y, node_id, depth):
        if (self.max_depth is not None and depth >= self.max_depth) or\
           (y.shape[0] < self.min_samples_split) or\
           (self.sufficient_share <= np.bincount(y).max() / y.shape[0]):
            
            class_ = np.bincount(y, minlength=self.num_class)
            self.tree[node_id] = (self.__class__.LEAF_TYPE, class_.argmax(),
                                  class_.astype(float) / y.size)
            return
            
        thr = np.array([self.__find_threshold(x[:, i], y) 
                        for i in self.get_feature_ids(x.shape[1])])
        best_id = thr[:, 0].argmin()
        best_thr = thr[best_id, 1]
        
        l_x,\
        r_x,\
        l_y,\
        r_y = self.__div_samples(x, 
                                 y, 
                                 self.__get_feature_ids_N(x.shape[1])[best_id],
                                 best_thr)
        
        if  l_x.shape[0] == 0 or\
            r_x.shape[0] == 0 or\
            best_thr is None:
            
            buf = np.bincount(y)
            self.tree[node_id] = (self.__class__.LEAF_TYPE, buf.argmax(),
                                  buf.astype(float) / y.size)
            return
        
        buf = np.bincount(y) / y.size
        if self.G_function == self.__gini:
            f = 1 - np.sum(buf ** 2)
            
        if self.G_function == self.__entropy:
            f = 1 - np.sum(buf * np.log(buf))
            
        if self.G_function == self.__misclass:
            f = 1 - np.max(buf)
        
        self.feature_importances_[best_id] += (f - thr[:, 0].min())
        
        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, best_id, best_thr)
        self.__fit_node(l_x, l_y, node_id * 2 + 1, depth + 1)
        self.__fit_node(r_x, r_y, node_id * 2 + 2, depth + 1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1], dtype=np.float)
        self.__fit_node(x, y, 0, 0) 
        self.feature_importances_ /= self.feature_importances_.sum()

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])
    
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [24]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2, criterion='gini')

In [25]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [26]:
%time clf.fit(X_train, y_train)

CPU times: user 12.5 ms, sys: 16 µs, total: 12.6 ms
Wall time: 10.6 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [27]:
%time my_clf.fit(X_train, y_train)

CPU times: user 52.1 ms, sys: 255 µs, total: 52.3 ms
Wall time: 48.5 ms


## Проверка качества работы на wine

In [28]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8885003885003885

In [29]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.7825396825396825

## Подготовка данных Speed Dating Data 

In [133]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')


In [134]:
to_drop = ['id', 'idg', 'condtn', 'wave', 'round', 'position',
           'positin1', 'order', 'partner', 'int_corr', 'pf_o_att', 
           'expnum', 'exphappy', 'pf_o_fun', 'pf_o_amb', 'pf_o_sin',
           'pf_o_int']

In [135]:
df = df.iloc[:, :90]
df = df.drop(to_drop, axis = 1)

import re
pattern = re.compile(".*\_o$")
for i in df.columns:
    if pattern.match(i):
        df = df.drop(i, axis = 1)

info = ['field_cd', 'mn_sat', 'tuition', 'income', 'career_c' ]
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:,info] = df.loc[:,info].fillna(0)


df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
            .drop(['gender'], axis=1).dropna()

df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
            .drop(['gender', 'match'], axis=1).dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)


for j in df_pair.columns:
    if  type(df_pair[j][df_pair[j].first_valid_index()]) is str:
        df_pair = df_pair.drop(j, axis=1)

df_pair = df_pair.dropna()
y = df_pair['match'].values
df_pair = df_pair.drop(['match'], axis=1)
X = df_pair.values

In [136]:
clf = DecisionTreeClassifier()
my_clf = MyDecisionTreeClassifier(criterion='gini')

In [137]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

## Проверка скорости работы на Speed Dating Data 

In [138]:
%time clf.fit(X_train, y_train)


CPU times: user 119 ms, sys: 0 ns, total: 119 ms
Wall time: 116 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [139]:
%time my_clf.fit(X_train, y_train)

CPU times: user 653 ms, sys: 45 µs, total: 653 ms
Wall time: 650 ms


## Проверка качества работы на Speed Dating Data

In [142]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5316154774277557

In [143]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5259118728902927

## Задание 3

## Задание 4

In [129]:
sk_i = np.argpartition(clf.feature_importances_, -10)[-10:]
sk_m = np.argpartition(my_clf.feature_importances_, -10)[-10:]

print('Sklearn clf:')
for i in sk_i:
    print(df_pair.columns[1:][i], ' ', clf.feature_importances_[i])
    
print('\nMy clf:')
for i in sk_m:
    print(df_pair.columns[1:][i], ' ', my_clf.feature_importances_[i])

Sklearn clf:
attr1_1   0.022555561402903875
attr3_1_f   0.022860759783623164
dining   0.023263493873007347
race_f   0.02401647956663927
concerts_f   0.024822033937606378
art_f   0.02769533816134711
hiking_f   0.02866327619447533
race   0.039472000839336704
yoga_f   0.0358516731725935
goal_f   0.03517779572079538

My clf:
age   0.020486481701630924
sinc2_1   0.043023870644607415
sinc3_1   0.020906988957419763
attr4_1   0.05614446136466173
theater_f   0.0688631457408275
concerts_f   0.14124619849504508
career_c   0.13286141387977096
amb1_1   0.1461808181119648
intel4_1_f   0.13855754948294863
pf_o_sha   0.11699689672699352


## Задание 5

In [146]:
p = {
    'n_estimators': range(1, 100, 10),
    'max_depth': range(1, 20),
    'min_samples_leaf': range(1, 20, 5),
    'criterion': ['gini', 'entropy']
}
clf = RandomizedSearchCV(RandomForestClassifier(), param_distributions=p, cv=5)
clf.fit(X_train, y_train)
clf.best_params_

{'n_estimators': 41,
 'min_samples_leaf': 1,
 'max_depth': 7,
 'criterion': 'gini'}

In [147]:
best_model = clf.best_estimator_
f1_score(y_pred=best_model.predict(X_test), y_true=y_test, average='macro')

0.4715586198433941